In [ ]:
import pandas as pd
import masst_utils
import glob
from tqdm import tqdm
import re

In [ ]:
parent_directory = r"D:\git\microbe_masst\output\piper_vs_microbiome\\"
min_cosine = 0.8


def create_empty_row():
    keys = ["masst_{}".format(special_masst.prefix) for special_masst in masst_utils.SPECIAL_MASSTS]
    return dict.fromkeys(keys, 0)


rows = dict()

for special_masst in masst_utils.SPECIAL_MASSTS:
    for file in tqdm(glob.glob(parent_directory + f"*counts_{special_masst.prefix}.tsv")):
        comp_id = re.search(r"_(\d+)_counts_" + special_masst.prefix, file).group(1)

        # get number of matches
        matches_df = pd.read_csv(file, sep='\t')
        if 'QC' in matches_df.columns:
            matches_df = matches_df[(matches_df['QC'] != 'Yes') & (matches_df['Blank'] != 'Yes')]

        matches_df = matches_df[matches_df['Cosine'] >= min_cosine]
        # exclude piper dataset
        matches_df = matches_df[matches_df['Dataset'] != "MSV000087844"]

        # get or create new
        row = rows.get(comp_id, create_empty_row())
        row["masst_{}".format(special_masst.prefix)] = len(matches_df)

        rows[comp_id] = row

df = pd.DataFrame(rows)
df

In [ ]:
df = df.transpose()
df.index.name = "rowID"
df

In [ ]:
df.to_csv(r"D:\git\microbe_masst\output\summary\piper_vs_microbiome.csv")

